<a href="https://colab.research.google.com/github/kokolight/MetacriticProject/blob/main/DataScienceProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataScience Project


## Creating the data set - Crawling 


### Main problem -
* Gaming companies develop games for years and cannot predict if their game will be sucessfull or not.

### Data -
Metacritic -

Description: Games ranking and reviews

Website: https://www.metacritic.com/

### Web scraping

### Data we scrape:
- Name - V
- Release Date - V
- Platforms - 
- Metascore - V
- Userscore
- Critic reviews
- Number of players
- Developer
- Publisher
- Maturaty Rating
- Genres
- Number of users ratings
- Number of critics ratings
- Number of awards
- Resolution

Possible:

- Number of positive/mixed/negative ratings
- Sound
- Online Playable (yes/no) 

In [169]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import threading

In [177]:
base_url = "https://www.metacritic.com/"
browse_url = base_url + 'browse/games/score/metascore/all/all/filtered'

month_dict = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May': '5', 'June':'6', 'July':'7',
                    'August':'8', 'September':'9', 'October':'10', 'November':'11', 'December':'12'}

soup = BeautifulSoup('html.parser')

def metacritic_games_data(num_of_pages):
  headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}

  names = []
  metascores = []
  userscores = []
  release_dates = []
  platforms = []
  hrefs = []

  for i in range(num_of_pages):
    page_url = browse_url+'page={0}'.format(i)
    req = requests.get(page_url, headers=headers)
    soup.append(bs4.BeautifulSoup(req.text))

  games_records = soup.find_all("td", class_="clamp-summary-wrap")

  for game in games_records:
    title_obj = game.find('a', class_="title")
    title =title_obj.text
    href = title_obj.get("href")
    meta_score = game.find('div', class_="metascore_w").text
    platform = game.find("span", class_="data").text.strip()
    names.append(title)
    hrefs.append(href)
    metascores.append(meta_score)
    platforms.append(platform)
    date=game.find("div", class_="clamp-details").find_all("span")[2].text.replace(",", "")
    date_splitted = date.split(" ")
    release_date="{day}-{month}-{year}".format(day = date_splitted[1], month = month_dict[date_splitted[0]] , year = date_splitted[2])
    release_dates.append(release_date)
    
    # # in game section
    # game_url = base_url+''+href
    # game_req = requests.get(game_url, headers=headers)
    # print(game_req)
    # game_soup = BeautifulSoup(game_req.text, 'html.parser')
    #user_score = game_soup.find("div", class_="metascore_w user").text
    #userscores.append(user_score)

  df = pd.DataFrame({'name': names, 'metascore': metascores, 'release_date': release_dates, 'platform': platforms, 'url': hrefs})
  return df

df = metacritic_games_data(10)


,name,metascore,release_date,platform,url
0,The Legend of Zelda: Ocarina of Time,99,23-11-1998,Nintendo 64,/game/nintendo-64/the-legend-of-zelda-ocarina-...
1,Tony Hawk's Pro Skater 2,98,20-9-2000,PlayStation,/game/playstation/tony-hawks-pro-skater-2
2,Grand Theft Auto IV,98,29-4-2008,PlayStation 3,/game/playstation-3/grand-theft-auto-iv
3,SoulCalibur,98,8-9-1999,Dreamcast,/game/dreamcast/soulcalibur
4,Grand Theft Auto IV,98,29-4-2008,Xbox 360,/game/xbox-360/grand-theft-auto-iv
...,...,...,...,...,...
895,Jet Grind Radio,94,30-10-2000,Dreamcast,/game/dreamcast/jet-grind-radio
896,Metal Gear Solid,94,21-10-1998,PlayStation,/game/playstation/metal-gear-solid
897,Grim Fandango,94,14-10-1998,PC,/game/pc/grim-fandango
898,Tom Clancy's Splinter Cell Chaos Theory,94,28-3-2005,Xbox,/game/xbox/tom-clancys-splinter-cell-chaos-theory


In [181]:
df.describe()

,name,metascore,release_date,platform,url
count,900,900,900,900,900
unique,75,6,87,16,100
top,Grand Theft Auto V,94,18-11-2014,PC,/game/playstation/metal-gear-solid
freq,45,315,18,144,9
